# Lunar Lander with Deep Q-Network (Part 2)

- toc: true 
- badges: true
- comments: false
- categories: [dqn, rl]

# Recap

> Note: The first part of this series can be found at [Part 1](/dqn/rl/2021/12/29/lunar_lander)

In the first part of this series, we implemented a DQN agent using the built in state output from the Gym environment. This state gives data on position, velociy, lander angle, and if either feet of the lander are touching the ground. For this second part, we'll up the ante and remove the state information, instead inferring state from the raw pixels of the game image.